
# OpenStreetMap with OSMnx

This example shows how to use OSMnx to download and model a street network
from OpenStreetMap, visualize centrality, and save the graph as a shapefile,
a GeoPackage, or GraphML.

OSMnx is a Python package to retrieve, model, analyze, and visualize
OpenStreetMap street networks as NetworkX MultiDiGraph objects. It can also
retrieve any other spatial data from OSM as geopandas GeoDataFrames. See
https://osmnx.readthedocs.io/ for OSMnx documentation and usage.


In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import os
import os.path
import multiprocessing as mp
import warnings
from functools import partial

import numpy as np
import networkx as nx
import osmnx as ox
import polars as pl
from tqdm.auto import tqdm
from matplotlib import pyplot as plt

from taxifare import data, boroughs

ox.settings.log_console=True
ox.settings.use_cache=True

In [ ]:
# ~ 6m 26s
# download street network data from OSM and construct a MultiDiGraph model

def load_map(base_map: str, simplified_map: str, tolerance: float) -> nx.MultiDiGraph:
    if os.path.exists(simplified_map):
        print(f"loading saved map {simplified_map}")
        H = ox.load_graphml(simplified_map)
        print(f"loaded saved map {simplified_map}")
        return H
    elif os.path.exists(base_map):
        print("Starting to simplify map. May require some time")
        H = ox.simplification._consolidate_intersections_rebuild_graph(
            ox.load_graphml(base_map), tolerance=tolerance, reconnect_edges=True)
        print(f"simplified base_map({base_map}) with tollerance = {tolerance}")
        ox.save_graphml(H, filepath=simplified_map)
        print(f"saved simplified graph: {simplified_map}")
        return H
    else:
        print("downloading graph. May require some time")
        G = ox.graph.graph_from_place("New York City, New York, USA")
        print("downloaded graph")
        G = ox.utils_graph.get_largest_component(G, strongly=True)
        print("Removed unconnected nodes")
        G = ox.add_edge_speeds(G)
        G = ox.add_edge_travel_times(G)
        print("added edge speeds and travel times")
        ox.save_graphml(G, filepath=base_map)
        print(f"saved base map: {base_map}")
        print("Starting to simplify map. May require some time")
        H = ox.simplification._consolidate_intersections_rebuild_graph(
            G, tolerance=tolerance, reconnect_edges=True)
        print(f"simplified with tollerance = {tolerance}")
        ox.save_graphml(H, filepath=simplified_map)
        print(f"saved simplified graph: {simplified_map}")
        return H

def plot_graph(G: nx.MultiDiGraph):
    ox.plot_graph(
        G, bgcolor="k", node_size=1, edge_linewidth=1, edge_color="#333333",
        figsize=(20, 20)
    )

In [ ]:
base_map = 'new_york_base.graphml'
tolerance = 0.00025
simplified_map = f'new_york_{tolerance}.graphml'

# load data ~1m 50s
G = load_map(base_map, simplified_map, tolerance)
df = data.load_data().fetch(500_000)

In [ ]:
plot_graph(G)
plt.show()

## Aproximated distance

Instead of calculating the distance for all points we calculate only for pairs of neigborhood

In [ ]:
kdtree = data.kdtree_from_graph(G)
G_undirected = G.to_undirected()

In [ ]:
travel_times_path = 'datasets/travel_times_simplified.pkl'

if os.path.exists(travel_times_path):
    with open(travel_times_path, 'rb') as f:
            warnings.warn('Distance matrix is a pickled file: use only '
                          'from trusted sources')
            travel_times = pickle.load(f)
    print(f'loaded from file {travel_times_path}')
else:
    travel_times = data.compute_graph_distances(G_undirected)